# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [94]:
#Libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [95]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [96]:
spaceship.shape

(8693, 14)

In [97]:
spaceship.isna().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [98]:
spaceship = spaceship.dropna()

In [99]:
spaceship.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

Now perform the same as before:
- Feature Scaling
- Feature Selection


In [100]:
#your code here
spaceship.drop(columns=['PassengerId', 'Name', 'Cabin'], inplace=True)
spaceship


,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True
...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False
8689,Earth,True,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,False
8690,Earth,False,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,True
8691,Europa,False,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False


In [101]:
spaceship_cat = spaceship[['HomePlanet', 'CryoSleep', 'VIP', 'Transported']]

In [102]:
encoder = LabelEncoder()
spaceship['HomePlanet'] = encoder.fit_transform(spaceship['HomePlanet'])
spaceship['Destination'] = encoder.fit_transform(spaceship['Destination'])
spaceship['CryoSleep'] = encoder.fit_transform(spaceship['CryoSleep'])                                    
spaceship['VIP'] = encoder.fit_transform(spaceship['VIP'])
spaceship['Transported'] = encoder.fit_transform(spaceship['Transported'])


In [103]:
spaceship

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,1,0,2,39.0,0,0.0,0.0,0.0,0.0,0.0,0
1,0,0,2,24.0,0,109.0,9.0,25.0,549.0,44.0,1
2,1,0,2,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0
3,1,0,2,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0
4,0,0,2,16.0,0,303.0,70.0,151.0,565.0,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...
8688,1,0,0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,0
8689,0,1,1,18.0,0,0.0,0.0,0.0,0.0,0.0,0
8690,0,0,2,26.0,0,0.0,0.0,1872.0,1.0,0.0,1
8691,1,0,0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,0


**Perform Train Test Split**

In [104]:
X = spaceship.drop(columns=['Transported'])
y = spaceship["Transported"]

In [105]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [106]:
normalizer = MinMaxScaler()

normalizer.fit(X_train)

MinMaxScaler()

In [107]:
X_train_norm = normalizer.transform(X_train)

X_test_norm = normalizer.transform(X_test)

In [108]:
X_train_norm = normalizer.transform(X_train)

X_test_norm = normalizer.transform(X_test)

In [109]:
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)
X_train_norm.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,0.5,0.0,0.0,0.316456,0.0,0.0,0.056116,0.000000,0.028650,0.030094
1,0.5,0.0,1.0,0.455696,0.0,0.0,0.088015,0.135232,0.124911,0.000049
2,1.0,1.0,0.0,0.430380,0.0,0.0,0.000000,0.000000,0.000000,0.000000
3,0.0,1.0,0.0,0.468354,0.0,0.0,0.000000,0.000000,0.000000,0.000000
4,0.0,1.0,0.5,0.278481,0.0,0.0,0.000000,0.000000,0.000000,0.000000


In [110]:
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns)
X_test_norm.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,0.5,1.0,1.0,0.367089,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,1.0,0.0,0.164557,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.0,1.0,1.0,0.632911,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.0,0.0,0.0,0.075949,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.0,0.0,1.0,0.468354,0.0,0.024458,0.000101,0.049049,0.000045,0.003344


**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [111]:
#your code here
bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100,
                               max_samples = 1000)

In [112]:
bagging_reg.fit(X_train_norm, y_train)
pred = bagging_reg.predict(X_test_norm)
print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", bagging_reg.score(X_test_norm, y_test))


MAE 0.27717227045051657
RMSE 0.37480898726779405
R2 score 0.43799056935368774


/Users/joaquinalborszamorano/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


- Random Forests

In [113]:
#your code here
forest = RandomForestRegressor(n_estimators=100,
                             max_depth=20)

In [114]:
forest.fit(X_train_norm, y_train)

RandomForestRegressor(max_depth=20)

In [115]:
pred = forest.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", forest.score(X_test_norm, y_test))

MAE 0.27347216881813485
RMSE 0.38972520355209594
R2 score 0.3923680571996935


/Users/joaquinalborszamorano/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


- Gradient Boosting

In [116]:
#your code here
gb_reg = GradientBoostingRegressor(max_depth=20,
                                   n_estimators=100)

In [117]:
gb_reg.fit(X_train_norm, y_train)

GradientBoostingRegressor(max_depth=20)

In [118]:
pred = gb_reg.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", gb_reg.score(X_test_norm, y_test))

MAE 0.2679410769329959
RMSE 0.44117314612463243
R2 score 0.2213509643784195


/Users/joaquinalborszamorano/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


- Adaptive Boosting

In [119]:
#your code here
ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=20),
                            n_estimators=100)

In [120]:
ada_reg.fit(X_train_norm, y_train)

AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=20),
                  n_estimators=100)

In [121]:
pred = ada_reg.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", ada_reg.score(X_test_norm, y_test))

MAE 0.26649272084536835
RMSE 0.4362572568456987
R2 score 0.23860689486357833


/Users/joaquinalborszamorano/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Which model is the best and why?

In [122]:
#comment here
# Best model is Bagging and Pasting because it has the highest R2 score with 0.4293 and the lowest RMSE with 0.3776